In [ ]:
# Import Section
try:
    from gekko import GEKKO
except:
    !pip install gekko
    # restart kernel if this doesn't import
    from gekko import GEKKO


from bokeh.settings import settings
# settings.resources = 'cdn'
# settings.resources = 'inline'
import warnings
import ipyvuetify as v
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from IPython.display import HTML, clear_output, display
from ARX import ARX
from _plots import df_plot
from _backend import *

pd.options.plotting.backend = "plotly"



warnings.filterwarnings('ignore')



class ARX_AddOn:
    colors = {
        'app_bar': '#007960',
        'controls_background': '#F6F6F6',
        'visualization_background': '#FFFFFF',
        'seeq_primary': '#007960',
    }
    
    additional_styles = widgets.HTML("""
        <style>
        .background_box { background-color:#007960 !important; } 
        .js-plotly-plot .plotly .modebar-btn[data-title="Produced with Plotly"] {display: none;}
        .vuetify-styles .theme--light.v-list-item .v-list-item__action-text, 
        .vuetify-styles .theme--light.v-list-item .v-list-item__subtitle {color: #212529;}
        .vuetify-styles .theme--light.v-list-item:not(.v-list-item--active):not(.v-list-item--disabled) 
        {color: #007960 !important;}
        .vuetify-styles .v-label {font-size: 14px;}
        .vuetify-styles .v-application {font-family: "Source Sans Pro","Helvetica Neue",Helvetica,Arial,sans-serif;}
        </style>""")
    v.theme.themes.light.success = '#007960'
    v.theme.themes.light.primary = '#007960'
    no_data_message = 'No plot available'


    
    
    def __init__(self, worksheet_url=None):
        
        if worksheet_url:
            try:
                sdl_notebook_url = f'{spy.utils.get_data_lab_project_url()}/dummy.ipynb?workbookId={spy.utils.get_workbook_id_from_url(worksheet_url)}&worksheetId={spy.utils.get_worksheet_id_from_url(worksheet_url)}'
                self.workbook_id, self.worksheet_id, self.workstep_id = get_workbook_worksheet_workstep_ids(sdl_notebook_url)
                self.worksheet_url = get_worksheet_url(sdl_notebook_url)
                self.df = pull_only_signals(self.worksheet_url)
#                 data = pd.read_csv('reactor.csv')
#                 data.set_index('Time', inplace=True)
#                 self.df = data
                self.df.dropna(inplace=True)
            except:
                self.df = pd.DataFrame()
        else:
            self.df = pd.DataFrame()

        self.model = None
        self.result_signal = pd.DataFrame()
        clear_output()

        self.graph = go.FigureWidget()
        
        self.get_url = create_textbox(label='Worksheet URL', placeholder='Please enter a worksheet URL', v_model='',
                                      color=self.colors['seeq_primary'], class_='mx-2', style_='', disabled=False)
        self.load_signal = v.Btn(color='success', children=['Signal from Worksheet'],
                                            target="_blank", disabled=False, loading=False,
                                            class_='mx-2', style_='text-transform: capitalize;')        
        
        # App layout
        self.hamburger_menu = HamburgerMenu()
        self.app = v.App(v_model=None, id='dummy-addon-app')
        
#         file = open( 'app_icon.png', 'rb')
#         image = file.read()
#         prism_logo = widgets.Image(value=image, format='png')
#         file.close()
        
#         title = v.Row(children=[prism_logo], class_='mr-3 ml-3 mt-2')
        self.appBar = v.AppBar(
            color=self.colors['app_bar'],
            dense=True,
            dark=True,
            children=[v.ToolbarTitle(children=['ARX Add-On'], class_='d-flex flex-row flex-wrap justify-left-space-between pr-2 pt-0'),
                      v.Spacer(),
                      v.Divider(vertical=True),
                      self.hamburger_menu])

        self.signal_mv = create_dropdowns(items=self.df.columns.to_list(),
                                         label='Select MV',
                                         color=self.colors['seeq_primary'],
                                         style_='max-width: 500px', class_='mx-4', multi=True)
        self.signal_cv = create_dropdowns(items=self.df.columns.to_list(),
                                         label='Select CV',
                                         color=self.colors['seeq_primary'],
                                         style_='max-width: 500px', class_='mx-4', multi=True)
        
        
        self.n_a = create_textbox(label='na', placeholder='Please enter na', v_model=2, color=self.colors['seeq_primary'], style_='max-width: 70px', class_='mx-2 pt-0 pl-2', disabled=True)
        self.n_b = create_textbox(label='nb', placeholder='Please enter nb', v_model=2, color=self.colors['seeq_primary'], style_='max-width: 65px', class_='mx-2 pt-0', disabled=True)
        self.n_k = create_textbox(label='nk', placeholder='Please enter nk', v_model=0, color=self.colors['seeq_primary'], style_='max-width: 65px', class_='mx-2 pt-0', disabled=True)

        
        self.adv_options= create_checkbox(label='Advanced Options', v_model=False, color=self.colors['seeq_primary'], style_='max-width: 500px', class_='mx-4 pt-0')

        
        
        self.create_model = v.Btn(color='success', children=['Identify Model'],
                                    target="_blank", disabled=True, loading=False,
                                    class_='mx-2', style_='text-transform: capitalize;')
        
        self.create_signals = v.Btn(color='success', children=['Signal to Workbench'],
                                    target="_blank", disabled=True, loading=False,
                                    class_='mx-2', style_='text-transform: capitalize;')

        
        
        self.workbook_address = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-space-between mx-2',
                                          style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                          children=[self.get_url, self.load_signal])
        
        self.dropdowns_container = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-space-between pr-0 pt-2',
                                          style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                          children=[self.signal_mv, self.signal_cv])
        
        self.orders_container = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-left-space-between pr-2 pt-0',
                                          style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                          children=[self.n_a, self.n_b, self.n_k])
        
        self.selection_container = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-space-between pr-0 pt-0',
                                          style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                          children=[self.dropdowns_container, self.adv_options])

        self.buttons_container = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-space-between pr-2 pt-2',
                                          style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                          children=[self.create_model, self.create_signals])
        
        self.r1 = v.Html(tag='div', class_='d-flex flex-column flex-wrap justify-space-between pr-0 pt-0',
                                style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                children=[self.selection_container, self.orders_container])
        
        self.container = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-space-between pr-0 pt-0',
                                style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                children=[self.r1, self.buttons_container])

        # controls bar
        self.controls = v.Html(tag='div', class_='d-flex flex-column pr-3 pl-3 pt-7',
                               style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                               children=[self.workbook_address, 'Choose signals and model order', self.container])

        # Visualization container
        self.visualization = v.Html(tag='div', id='plotly-heatmap',
                                    style_=f"background-color: {self.colors['visualization_background']};"
                                           f"border:2px solid {self.colors['controls_background']};",
                                    children=[self.graph])      
    
        self.create_displayed_fig(df_plot(pd.DataFrame))

    
    def identify_system(self, *_):
        self.model = ARX()
        self.model.identify(self.df[self.signal_mv.v_model], self.df[self.signal_cv.v_model], pd.DataFrame(self.df.index), int(self.n_a.v_model), int(self.n_b.v_model), int(self.n_k.v_model))
#         self.result_signal = self.model.predict(self.df[self.signal_mv.v_model], self.df[self.signal_cv.v_model])
        self.result_signal = self.model.forecast(self.df[self.signal_mv.v_model], self.df[self.signal_cv.v_model].head(max(int(self.n_b.v_model)+int(self.n_k.v_model), int(self.n_a.v_model))))
        self.result_signal.set_index(self.df.index, inplace=True)
        self.result_signal[self.signal_cv.v_model] = self.df[self.signal_cv.v_model]

        self.result_signal = self.result_signal.iloc[:, ::-1]

        fig = df_plot(self.result_signal)
        self.create_displayed_fig(fig)
        if fig:
            self.create_signals.disabled = False
        else:
            self.create_signals.disabled = True
        
    def create_displayed_fig(self, fig):
        if fig is None:
            self.graph = self.no_data_message
            return
        self.graph = go.FigureWidget(fig)
        self.visualization.children = [self.graph]

        
    def update_display(self, *_):
        signal_mv = list(self.df.columns)
        signal_cv = list(self.df.columns)

        self.signal_mv.items = [item for item in self.df.columns if item not in self.signal_cv.v_model]
        self.signal_cv.items = [item for item in self.df.columns if item not in self.signal_mv.v_model]
        if len(self.signal_mv.v_model)>0 and len(self.signal_cv.v_model)>0:
            self.create_model.disabled = False
        else:
            self.create_model.disabled = True
            self.create_signals.disabled = True

            
    def disp_advanced_options(self, *_):

        if self.adv_options.v_model:
            for item in range(len(self.orders_container.children)):
                self.orders_container.children[item].disabled = False
        else:
            for item in range(len(self.orders_container.children)):
                self.orders_container.children[item].disabled = True
                
    def push_to_seeq(self, *_):
        push_signal(self.result_signal, self.workbook_id, 'From ARX Add-on')
        
        
    def run(self):
        # noinspection PyTypeChecker
        display(HTML("<style>.container { width:100% !important; }</style>"))
        self.app.children = [self.appBar, self.controls, self.visualization, self.additional_styles]
        self.load_signal.on_event('click', self.reset_url)
        self.signal_mv.on_event('change', self.update_display)
        self.signal_cv.on_event('change', self.update_display)
        self.adv_options.on_event('change', self.disp_advanced_options)
        self.create_model.on_event('click', self.identify_system)
        self.create_signals.on_event('click', self.push_to_seeq)
        return self.app
    
    
    
    
    def reset_url(self, *_):
#         clear_output()
        
        worksheet_url = self.get_url.v_model
        if worksheet_url:
            sdl_notebook_url = f'{spy.utils.get_data_lab_project_url()}/dummy.ipynb?workbookId={spy.utils.get_workbook_id_from_url(worksheet_url)}&worksheetId={spy.utils.get_worksheet_id_from_url(worksheet_url)}'
            self.workbook_id, self.worksheet_id, self.workstep_id = get_workbook_worksheet_workstep_ids(sdl_notebook_url)
            self.worksheet_url = get_worksheet_url(sdl_notebook_url)
            self.df = pull_only_signals(self.worksheet_url)
#             data = pd.read_csv('reactor.csv')
#             data.set_index('Time', inplace=True)
#             self.df = data
            self.df.dropna(inplace=True)
        else:
            self.df = pd.DataFrame()
        self.result_signal = pd.DataFrame()
        fig = df_plot(self.result_signal)
        self.create_displayed_fig(fig)
        self.signal_mv.v_model = []
        self.signal_cv.v_model = []
        self.update_display()


    
    
    
    
    
class HamburgerMenu(v.Menu):
    def __init__(self, **kwargs):
        self.hamburger_button = v.AppBarNavIcon(v_on='menuData.on')
        self.help_button = v.ListItem(value='help',
                                      ripple=True,
                                      href='mailto: support@company.com?subject=MyAddOn Feedback',
                                      children=[v.ListItemAction(class_='mr-2 ml-0',
                                                                 children=[v.Icon(color='#212529',
                                                                                  children=['fa-life-ring'])]),
                                                v.ListItemActionText(children=[f'Send Support Request'])
                                                ])
        self.items = [v.Divider(), self.help_button, v.Divider()]

        super().__init__(offset_y=True,
                         offset_x=False,
                         left=True,
                         v_slots=[{
                             'name': 'activator',
                             'variable': 'menuData',
                             'children': self.hamburger_button,
                         }]
                         ,
                         children=[
                             v.List(children=self.items)
                         ]
                         , **kwargs)
        
        
        
def create_dropdowns(items: list, label='', color='', v_model=[], class_='', style_='', multi=False):
    return v.Select(multiple=multi,
                    label=label,
                    items=items,
                    dense=True,
                    outlined=True,
                    color=color, filled=True,
                    item_color='primary',
                    v_model=v_model,
                    style_=style_,
                    class_=class_)


def create_checkbox(label='', color='', v_model='', class_='', style_=''):
    return v.Checkbox(label=label,
                      v_model=v_model,
                      dense=True,
                      outlined=True,
                      color=color, filled=True,
                      item_color='primary',
                      style_=style_,
                      class_=class_)


def create_textbox(label='', placeholder='', v_model='2', color='', class_='', style_='', disabled=False):
     return v.TextField(label=label,
                        placeholder=placeholder,
                        v_model=v_model,
                        dense=True,
                        outlined=True,
                        color=color,
                        filled=True,
                        style_=style_,
                        class_=class_,
                        disabled = disabled,
                        loading=False)

In [ ]:
c = ARX_AddOn()
c.run()



In [ ]:
# url = 'http://prism-seeq-server:34216/3885E5D7-3B9C-4F90-90FE-4E91F695BCF0/workbook/4FB6FCB6-FB06-47A5-8C2E-8B6B5D02B9D2/worksheet/80921B9D-AB5A-4AA0-9DFB-F8E9348E92A2'
# worksheet = spy.utils.get_analysis_worksheet_from_url(url)
# start = worksheet.display_range['Start']
# end = worksheet.display_range['End']

# search_df = spy.search(url, estimate_sample_period=worksheet.display_range, quiet=True)

# # search_signals_df = search_df[search_df['Type'].str.contains('l')]

# df = spy.pull(search_df, start=start, end=end, grid='auto', header='ID', quiet=True,
#               status=spy.Status(quiet=True))

# df.columns = df.query_df['Name']



In [ ]:
# df = df.dropna()
# df
# df.to_csv('df.csv')
# # df[df['Manual Condition 1']==1]
# # capsule_df  = search_df[search_df['Type'].str.contains('CalculatedCondition')]

In [ ]:
# capsule_df.Name